In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input # MobileNetV2
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [74]:
PATH = "mias"
IMAGE_SIZE = 224

In [75]:
dataset = image_dataset_from_directory(
    PATH,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=322
)

Found 322 files belonging to 2 classes.


In [76]:
dataset.class_names

['abnormal', 'normal']

In [77]:
dataset = dataset.cache()

In [78]:
X = [element[0] for element in dataset.as_numpy_iterator()]
y = [element[1] for element in dataset.as_numpy_iterator()]

In [79]:
X = X[0]
y = y[0]

In [80]:
sum(y==1), sum(y==0)

(207, 115)

In [81]:
X.shape

(322, 224, 224, 3)

In [82]:
X_res = X.reshape(322, IMAGE_SIZE * IMAGE_SIZE * 3)
X_res.shape

(322, 150528)

In [83]:
y.shape

(322,)

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y, test_size=0.3, random_state=0)

In [100]:
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.15, random_state=0)

In [85]:
X_train.shape, X_test.shape

((225, 150528), (97, 150528))

In [86]:
sum(y_train==0), sum(y_train==1)

(78, 147)

In [87]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [89]:
X_train_res.shape, y_train_res.shape

((294, 150528), (294,))

In [93]:
sum(y_train_res == 0), sum(y_train_res == 1)

(147, 147)

In [94]:
X_train_res = X_train_res.reshape(294, IMAGE_SIZE, IMAGE_SIZE, 3)

In [95]:
X_train_res.shape

(294, 224, 224, 3)

In [97]:
def get_training_model(trainable=False):
    
    IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
    BASE_MODEL = MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
    
    # We will set it to both True and False
    BASE_MODEL.trainable = trainable
    
    # Construct the head of the model that will be placed on top of the
    # the base model
    class_head = BASE_MODEL.output
    class_head = GlobalAveragePooling2D()(class_head)
    class_head = Dense(512, activation="relu")(class_head)
    class_head = Dropout(0.5)(class_head)
    class_head = Dense(2, activation="softmax")(class_head)

    # Create the new model
    classifier = Model(inputs=BASE_MODEL.input, outputs=class_head)
    return classifier

In [98]:
early_stopping = EarlyStopping(
                        monitor='val_loss', 
                        min_delta=0, 
                        patience=0, 
                        verbose=0, 
                        mode='auto', 
                        baseline=None, 
                        restore_best_weights=True
                    )

In [101]:
model = get_training_model(trainable=True)

model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="adam",
              metrics=["accuracy"])

history = model.fit(X_train_res, y_train_res , validation_data=(X_valid, y_valid), 
                    verbose = 1, epochs = 2, 
                    shuffle=True,
                    batch_size=18,
                    callbacks=[early_stopping])

Epoch 1/2
10/10 [==============================] - ETA: 0s - loss: 1.0515 - accuracy: 0.5680 

ValueError: in user code:

    File "G:\Python3\lib\site-packages\keras\engine\training.py", line 1727, in test_function  *
        return step_function(self, iterator)
    File "G:\Python3\lib\site-packages\keras\engine\training.py", line 1713, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "G:\Python3\lib\site-packages\keras\engine\training.py", line 1701, in run_step  **
        outputs = model.test_step(data)
    File "G:\Python3\lib\site-packages\keras\engine\training.py", line 1665, in test_step
        y_pred = self(x, training=False)
    File "G:\Python3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "G:\Python3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "model_1" is incompatible with the layer: expected shape=(None, 224, 224, 3),